In [1]:
import torch
print(torch.__version__)  # should be >= 1.13.1
print("CUDA available?", torch.cuda.is_available())


2.6.0
CUDA available? False


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.datasets import fetch_openml
import numpy as np

# --------------------------------------------
# ⚙️ Device Setup
# --------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --------------------------------------------
# 📥 Load MNIST from OpenML (ID = 554)
# --------------------------------------------
mnist = fetch_openml(name="mnist_784", version=1)
X, y, _, _ = mnist.get_data(target=mnist.default_target_attribute, dataset_format='array')

# Preprocessing
X = X.reshape(-1, 1, 28, 28).astype(np.float32) / 255.0
y = y.astype(np.int64)

# Convert to tensors
X_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)

# Dataset
full_dataset = TensorDataset(X_tensor, y_tensor)

# 80/20 split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# --------------------------------------------
# 🧠 CNN Model
# --------------------------------------------
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(-1, 64 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# --------------------------------------------
# 🧪 Training & Evaluation Functions
# --------------------------------------------
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()

    return total_loss / len(loader.dataset), correct / len(loader.dataset)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

# --------------------------------------------
# 🚀 Run Training
# --------------------------------------------
model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
criterion = nn.NLLLoss()

for epoch in range(1, 6):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)
    print(f"Epoch {epoch}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")

# --------------------------------------------
# ✅ Final Accuracy Report
# --------------------------------------------
print("\nEvaluating on 20% test split from OpenML...")
final_loss, final_acc = evaluate(model, val_loader, criterion)
print(f"Final Validation Accuracy: {final_acc:.4f}")



Using device: cpu


AttributeError: get_data